# NLP - PROCESSAMENTO DE LINGUAGEM NATURAL

In [ ]:
# PARA TRABALHAR COM O GOOGLE COLAB É NECESSÁRIO:
# !apt-get update -qq
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
# !tar xf spark-3.1.2-bin-hadoop2.7.tgz
# !pip install -q findspark

import os
import findspark

os.environ["SPARK_HOME"] = "D:/spark-3.5.5-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

import pandas as pd

### Obs: para este projeto estamos trabalhando com o google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
SPARK = spark.read.csv('/content/drive/ml/imdb-reviews-pt-br.csv', escape='\"', inferSchema=True, header=True)

In [ ]:
# LEMATIZAÇÃO E STEMING (REMOÇÃO DE PALAVRAS DESNECESSARIAS PARA O PROCESSAMENTO COMO PRONOMES, PREPOSIÇÕES "STOP-WORDS", ETC)
# PARA MELHORAR O RECURSO VISUAL UTILIZAREMOS O WORDCLOUD, DESSA MANEIRA VAMOS ENXERGAR QUAIS AS PALAVRAS MAIS UTILIZADAS EM UM TEXTO
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# INICAMOS PEGANDO APENAS UMA AMOSTRA DE 10% PARA AJUDAR O PROCESSAMENTO (collect é mt pesado)
AMOSTRA = SPARK.select('text_pt').sample(fraction=0.1, seed=101)
DATA = [texto['text_pt'] for texto in AMOSTRA.collect()]

wordcloud = WordCloud(collocations=False, 
                      prefer_horizontal=1,
                      width=1000,
                      height=600).generate(str(DATA))

plt.figure(figsize=(20,8))
plt.imgshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# REMOÇÃO DE CARACTERES ESPECIAIS
import string
print(string.punctuation)

SPARK = SPARK.withColumn('texto_regex', f.regexp_replace('text_en', "[\$#,\"!%&'()*+,-./:;<=>?@[\]^_`{|}~\\\\]"), "")
SPARK = SPARK.withColumn('texto_limpo', f.trim(SPARK.texto_regex))

#### TOKENIZAÇÃO DO TEXTO

In [ ]:
# UM TEXTO QUALQUER ===>  [UM][TEXTO][QUALQUER]
# TEXTO COMPOSTO GUARDA-CHUVA ====> [TEXTO][COMPOSTO][GUARDA-CHUVA]
from pyspark.ml.features import Tokenizer

tokenizer = Tokenizer(inputCol='texto_limpo', outputCol='tokens')
SPARK_TOKENIZADO = tokenizer.transform(SPARK)

In [ ]:
from pyspark.sql.types import IntegerType

count_token = f.udf(lambda x: len(x), IntegerType())
SPARK_TOKENIZADO.select('texto_limpo', 'tokens').withColumn('qtd_tokens', count_token(f.col('tokens')))

#### REMOVENDO STOP-WORDS

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# stop_A = stopwords('portuguese')

from pyspark.ml.features import StopWordsRemover
# stop_B = StopWordsRemover.loadDefaultStopWords('portuguese')

remover = StopWordsRemover(inputCol='tokens', outputCol='texto_final')
SPARK_TOKENIZADO_WORDS = remover.transform(SPARK_TOKENIZADO)
SPARK_TOKENIZADO_WORDS.select('texto_limpo', 'tokens').withColumn('qtd_tokens', count_token(f.col('tokens')))\
                                                      .withColumn('qtd_tokens_limpos', count_token(f.col('texto_final')))

#### BAG OF WORDS - VETORIZAÇÃO 

In [ ]:
# importante ressaltar que a ordem na qual as palavras ficam no indice da vetorização
# é referente ao model.vocabulary e não a ordem da string em si
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol='texto_final', outputCol='CountVec')
model = cv.fit(SPARK_TOKENIZADO_WORDS)
CV_FEATURES = model.transform(SPARK_TOKENIZADO_WORDS)

######################
# HASHING TF - ALTERNATIVA AO BAG OF WORDS COM PROCESSAMENTO MELHOR
from pyspark.ml.feature import HashingTF

hashing = HashingTF(inputCol='texto_final', outputCol='HashingTF')
hashing.setNumFeatures(50) #Limitando a quantidade de palavras para executar

SPARK_HASHING = hashing.transform(CV_FEATURES)

#### TF-IDF - PONDERAÇÃO DE PESOS

In [ ]:
from pyspark.ml.feature import IDF

idf = IDF(inpuCol='HashingTF', outputCol='features')
idfModel = idf.fit(SPARK_HASHING)
SPARK_IDF = idfModel.transform(SPARK_HASHING)

In [ ]:
# TRATANDO COLUNA DE SENTIMENTO STRING PARA BINARIO
from pyspark.ml.feature import StringIndexer

string_indexer = StringIndexer(inputCol='sentiment', outputCol='label')
SPARK = string_indexer.fit(SPARK).transform(SPARK)

#### PIPELINE - UNINDO AS TRANSFORMAÇÕES

In [ ]:
from pyspark.ml import Pipeline

#### UTILIZANDO A CLASSIFICAÇÃO PELA ARVORE DE DECISÃO
from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(featuresCol='features', labelCol='label', maxDepth=10)

tokenizer = Tokenizer(inputCol='texto_limpo', outputCol='tokens')
remover = StopWordsRemover(inputCol='tokens', outputCol='texto_final')
hashing = HashingTF(inputCol='texto_final', outputCol='HashingTF', numFeatures=1000)
idf = IDF(inpuCol='HashingTF', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, remover, hashing, idf, dtc])

DF_FINAL = pipeline.fit(SPARK).transform(SPARK)

#### TREINO E ANALISE DA ARVORE DE DECISÃO

In [ ]:
TREINO, TESTE = SPARK.randomSplit([0.7, 0.3], seed=101)

MODELO_DTC = pipeline.fit(TREINO)
PREVISOES_DTC = MODELO_DTC.transform(TESTE)

#### AVALIANDO AS MÉTRICAS PELO MULTICLASS EVALUATOR (ACURACIA)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()
print("Acurácia: %f" % evaluator.evaluate(PREVISOES_DTC, {evaluator.metricName: 'accuracy'}))